In [31]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import re
import geopandas as gpd

In [32]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

df = pd.read_csv('KeithsAddress - Sheet1.csv')
df['full_address'] = df['address'] + 'Brooklyn, New York, NY'
df

,address,full_address
0,17 Devoe Street,"17 Devoe StreetBrooklyn, New York, NY"
1,165 Central Avenue,"165 Central AvenueBrooklyn, New York, NY"
2,198 Scholes Street,"198 Scholes StreetBrooklyn, New York, NY"
3,11 Gunther Place,"11 Gunther PlaceBrooklyn, New York, NY"
4,194 Ralph Avenue,"194 Ralph AvenueBrooklyn, New York, NY"
...,...,...
107,132A Stanhope Street,"132A Stanhope StreetBrooklyn, New York, NY"
108,268 Metropolitan Avenue,"268 Metropolitan AvenueBrooklyn, New York, NY"
109,236 Meserole Street,"236 Meserole StreetBrooklyn, New York, NY"
110,430 Albee,"430 AlbeeBrooklyn, New York, NY"


In [33]:
# df = dsf[0]
# df.columns = df.iloc[0]
# df = df.drop(df.index[0])
# ### This next line gets ride of the Nan ###
# df = df.iloc[: , 1:]
# df = df.replace(np.nan, 'Not Available', regex=True)
# df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
#                         'Price /\rSq. Ft.':'price_per_sqft'})
# df['geo_address'] = df['building_address_city']
# df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
# df = df.replace('\n',' ',regex=True)
# df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

In [34]:
# result = pd.merge(df,df2,left_index=True, right_index=True)
# df = result

In [35]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [ ]:
df['loc'] = df['full_address'].apply(geolocator.geocode, timeout=10)

In [ ]:
df['loc']

In [ ]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [ ]:
df.columns

In [ ]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [ ]:
df

In [ ]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11, tiles='CartoDB positron')
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'All Year\'s Real Estate Holdings')

marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["address"], r["altitude"])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Address: {bold_start}{info[0]}{bold_end}{new_line}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))
m

In [ ]:
url = ("https://raw.githubusercontent.com/python-visualization/folium/master/examples/data")
nybb = f"{url}/nybb.zip"
boros = gpd.read_file(nybb)

# colors added
fill_color = ['orange','yellow','purple','red','blue']
boros['fill_color'] = fill_color

from folium.features import GeoJsonTooltip

tooltip = GeoJsonTooltip(
    fields=["BoroName"],
    labels=True
    )

folium.GeoJson(boros,
               style_function=lambda x: {'fillColor':x['properties']['fill_color']},
               tooltip=tooltip
               ).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m

In [ ]:
m.save('index.html')